In [ ]:
import json
import os
import cv2
import albumentations as A 
import matplotlib.pyplot as plt
import numpy as np

BOX_COLOR = (255, 0, 0) # Red
TEXT_COLOR = (255, 255, 255) # White
CROP_WIDTH = 2560
CROP_HEIGHT = 2560

transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.Rotate(limit=45, p=0.5),  
    A.BBoxAndKeypointsSafeRandomCrop(p=1.0),
    A.Equalize(mode = "cv"),
    A.GaussNoise (var_limit=(10.0, 50.0), mean=0, per_channel=True, always_apply=False, p=0.5),
    A.OneOf([
                A.HueSaturationValue(hue_shift_limit=20, sat_shift_limit=30, p=0.5),
                A.RandomBrightnessContrast(brightness_limit=0.2, 
                                           contrast_limit=0.2, p=0.5),
            ],p=0.7),
    A.Blur(blur_limit=(3, 7), p=0.3), 
    A.RandomGamma(gamma_limit=(80, 120), p=0.5)
], bbox_params = A.BboxParams(format='coco', min_visibility=0.3), keypoint_params = A.KeypointParams(format='xy', remove_invisible=False, label_fields=['class_labels']))

input_folder = './images/'
output_folder = './images_augmented'
vis_folder = './images_augmented_with_anno'

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

category_id = [1]
category_id_to_name = {1: 'ear'}
class_labels = [
        "\u80be\u4e0a\u817a",
        "\u8033\u5c16",
        "\u80c3",
        "\u773c",
        "\u53e3",
        "\u809d",
        "\u5bf9\u5c4f\u5c16",
        "\u5fc3",
        "\u80ba",
        "\u80ba2",
        "\u8180\u80f1",
        "\u813e",
        "\u89d2\u7a9d\u4e2d",
        "\u795e\u95e8",
        "\u80be",
        "\u8033\u95e8",
        "\u542c\u5bab",
        "\u542c\u4f1a",
        "\u80a9",
        "\u6241\u6843\u4f53",
        "\u8170\u9ab6\u690e"
      ]

with open('./train_coco.json', 'r') as f:
    data = json.load(f)

images = data['images']
annotations = data['annotations']

new_data = {}
new_images = []
new_annotations = []

for image in images:
    image_id = image['id']
    image_path = os.path.join(input_folder, image['file_name'])
    image_bbox = annotations[image_id]['bbox']
    image_category = annotations[image_id]['category_id']
    image_keypoints = annotations[image_id]['keypoints']

    key_points = [(image_keypoints[i], image_keypoints[i + 1]) for i in range(0, len(image_keypoints), 3)]

    image_bboxes = [image_bbox + [category_id_to_name[image_category]]]
    x_min, y_min, w, h = image_bbox
    crop_bbox = [int(x_min), int(y_min), int(x_min + w), int(y_min + h)]

    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

    transformed = transform(image=img, bboxes=image_bboxes, keypoints = key_points, class_labels = class_labels, cropping_bbox = crop_bbox)

    vis_image = visualize(
    transformed['image'],
    transformed['bboxes'],
    transformed['keypoints']
    )
    vis_image = cv2.cvtColor(vis_image, cv2.COLOR_BGR2RGB)

    transformed_image = transformed['image']
    transformed_image = cv2.cvtColor(transformed_image, cv2.COLOR_BGR2RGB)
    transformed_keypoints = transformed['keypoints']
    transformed_keypoints = [coord for tup in transformed['keypoints'] for coord in tup + (2,)] 
    transformed_keypoints = [int(x) for x in transformed_keypoints]

    transformed_bbox = [int(x) for x in transformed['bboxes'][0][:4]]
    transformed_area = transformed_bbox[2]*transformed_bbox[3]

    new_annotations.append({
        "category_id": 1,
        "segmentation": [],
        "iscrowd": 0,
        "image_id": image_id,
        "id": image_id,
        "bbox": transformed_bbox,
        "area": transformed_area,
        "num_keypoints": len(transformed['keypoints']),
        "keypoints": transformed_keypoints
    })

    new_images.append({
        "file_name": image['file_name'],
        "height": transformed_image.shape[0],
        "width": transformed_image.shape[1],
        "id": image_id
    })
    
    output_path = os.path.join(output_folder, image['file_name'])
    cv2.imwrite(output_path , transformed_image)

    vis_path = os.path.join(vis_folder, image['file_name'])
    cv2.imwrite(vis_path , vis_image)

In [ ]:
new_data = {}
new_data = {
    "categories": data['categories'],
    "images": new_images,
    "annotations": new_annotations
}
with open("train_augmented_coco.json", "w") as json_file:
    json.dump(new_data, json_file, indent=4)